In [1]:
# yelp scraping project
import requests
import json
import pandas as pd

from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import re

In [13]:
# create input variables

search = input("What would you like to search:")
zipcode = input("where would you like to search:")

What would you like to search:men's hari
where would you like to search:94526


In [4]:
# default web search to concatenate
default = "https://www.yelp.com/search?find_desc="
locationUrl = "&find_loc="
end = "&ns=1"

# empty lists to append to
url = []
business_name = []
business_category = []
yelp_rating = []
review_count = []
price_range = []
price_category = []

address = []
phone = []
website = []

In [5]:
# function to search input parameters on Yelp
def search_yelp(search, zipcode):
    
    keyword = search;
    zipcode = zipcode;
    query_url = f"{default}{search}{locationUrl}{zipcode}{end}";
    response = requests.get(query_url)
    soup = BeautifulSoup(response.text, 'html.parser');
    for link in soup.find_all('a', class_="biz-name"):
        url.append("https://www.yelp.com"+link.get('href'))
    url.pop(0)
    

In [6]:
# function to scrape yelp data on each page
def scrape_yelp(url):
    for i in url:
        response = requests.get(i)
        soup = BeautifulSoup(response.text, 'html.parser')
        business_name.append(soup.find('h1', attrs={'class':'biz-page-title'}).text.strip())
        business_category.append(soup.find(attrs={'class': 'category-str-list'}).text.strip())
        yelp_rating.append(soup.find(attrs={"class":"i-stars"})["title"])
        review_count.append(soup.find(attrs={'class': 'review-count rating-qualifier'}).text.strip())
        price_range.append(soup.find(attrs={'class': 'business-attribute price-range'}).text.strip())
        price_category.append(soup.find(attrs={'class': 'nowrap price-description'}).text.strip())
        address.append(soup.find(attrs={'class': 'street-address'}).text.strip())
        phone.append(soup.find(attrs={'class': 'biz-phone'}).text.strip())
        
        if soup.find("a", href=lambda href: href and "biz_redir?" in href) is not None:
            website.append(soup.find("a", href=lambda href: href and "biz_redir?" in href).text.strip())
        else:
            website.append('no website')

In [11]:
# create a dataframe with yelp data from yelp scrape

def create_table():
    data = {
    'BusinessName' : business_name,
    'BusinessCategory' : business_category,
    'YelpRating' : yelp_rating,
    'ReviewCount' : review_count,
    'PriceRange($)' : price_range,
    'PriceCategory': price_category,
    'Address' : address, 
    'Phone' : phone,
    'Website' : website
        
    }
    # return data
    df = pd.DataFrame.from_dict(data)
    df = df[['BusinessName','BusinessCategory','YelpRating','ReviewCount','PriceRange($)', 'PriceCategory','Address',
            'Phone', 'Website']]
    df.to_csv(f'data/yelp_{search}{zipcode}.csv')
    return df

In [8]:
search_yelp(search, zipcode)

In [9]:
scrape_yelp(url)

In [14]:
create_table()

,BusinessName,BusinessCategory,YelpRating,ReviewCount,PriceRange($),PriceCategory,Address,Phone,Website
0,Omar the Barber,Barbers,4.5 star rating,303 reviews,$,Inexpensive,"13925 Yale AveKija Hair SalonIrvine, CA 92620",(714) 618-1133,omarthebarber.com
1,Danckuts,Barbers,4.5 star rating,303 reviews,$$,Moderate,Located inTrabuco Plaza,(949) 400-0444,danckuts.com
2,Stonecreek Plaza Barber Shop,Barbers,5.0 star rating,131 reviews,$,Inexpensive,Located inStonecreek Plaza,(949) 786-6404,stonecreekplazabarbershop.com
3,Justin’s Fades & Hairstyles,Barbers,5.0 star rating,44 reviews,$$,Moderate,"4330 Barranca PkwySte 105Irvine, CA 92604",(949) 973-1601,justins-fadeshairstyles.busin…
4,Roosters Men’s Grooming Center,Barbers,4.0 star rating,167 reviews,$$,Moderate,"5405 Alton Pkwy Ste DIrvine, CA 92604",(949) 551-5909,roostersmgc.com
5,Made Man Salon,"Men's Hair Salons,\n Waxing...",5.0 star rating,23 reviews,$$,Moderate,"2801 El Camino RealSte 28Tustin, CA 92782",(949) 290-8122,mademansalon.com
6,Hair Days,Hair Salons,4.5 star rating,160 reviews,$$,Moderate,"14231 Jeffrey RdIrvine, CA 92620",(949) 733-3700,hairdaysbeauty.com
7,The Sharper Look,"Hair Salons,\n Skin Care,\n...",5.0 star rating,63 reviews,$$,Moderate,"15333 Culver DrSte 21Irvine, CA 92604",(949) 829-3288,thesharperlook.com
8,Bespoke Tailored Men’s Haircare,Men's Hair Salons,5.0 star rating,25 reviews,$$,Moderate,"15333 Culver DrSte 200Irvine, CA 92604",(949) 735-1529,no website
9,Sand Canyon Barber,Barbers,3.5 star rating,91 reviews,$$,Moderate,"6642 Irvine Center DrIrvine, CA 92618",(949) 727-3000,sandcanyonbarbershop.com
